In [0]:
%run "../includes/configuration"

In [0]:
df = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
from pyspark.sql.functions import desc,sum, when,count

In [0]:
df = df \
    .groupBy(df.race_year, df.driver_name, df.team) \
    .agg(
        sum(df.points).alias("total_points"),
        count(when(df.position == 1, True )).alias("wins")
    ) \
    .orderBy(desc("total_points"))

In [0]:
display(df.filter(df.race_year == 2020))

race_year,driver_name,team,total_points,wins
2020,Lewis Hamilton,Mercedes,347.0,11
2020,Valtteri Bottas,Mercedes,223.0,2
2020,Max Verstappen,Red Bull,214.0,2
2020,Sergio Pérez,Racing Point,125.0,1
2020,Daniel Ricciardo,Renault,119.0,0
2020,Carlos Sainz,McLaren,105.0,0
2020,Alexander Albon,Red Bull,105.0,0
2020,Charles Leclerc,Ferrari,98.0,0
2020,Lando Norris,McLaren,97.0,0
2020,Lance Stroll,Racing Point,75.0,0


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

In [0]:
df = df.withColumn("rank", rank().over(Window.partitionBy(df.race_year).orderBy(desc("total_points"), df.wins)))

In [0]:
display(df.filter(df.race_year == 2020))

race_year,driver_name,team,total_points,wins,rank
2020,Lewis Hamilton,Mercedes,347.0,11,1
2020,Valtteri Bottas,Mercedes,223.0,2,2
2020,Max Verstappen,Red Bull,214.0,2,3
2020,Sergio Pérez,Racing Point,125.0,1,4
2020,Daniel Ricciardo,Renault,119.0,0,5
2020,Carlos Sainz,McLaren,105.0,0,6
2020,Alexander Albon,Red Bull,105.0,0,6
2020,Charles Leclerc,Ferrari,98.0,0,8
2020,Lando Norris,McLaren,97.0,0,9
2020,Lance Stroll,Racing Point,75.0,0,10


In [0]:
df.write.mode("overwrite").parquet(f"{presentation_folder_path}/driver_standings")

#df.write.mode("overwrite").saveAsTable("f1_presentation.driver_standings")